In [2]:
!pip install jax jaxlib

/opt/anaconda3/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [4]:
# Import necessary libraries
import jax
import jax.numpy as jnp
from jax import profiler

# Function to compute the MSE loss with potential memory issues
def mse_loss_one_batch(mat_u, mat_v, rows, columns, ratings):
    # This could lead to high memory usage
    estimator = -(mat_u @ mat_v)[(rows, columns)]
    loss = jnp.mean((estimator - ratings) ** 2)
    return loss

# Function to inspect memory usage
def run_memory_inspection(mat_u, mat_v, rows, columns, ratings):
    # Create a log directory for the profiler
    log_dir = '/tmp/jax_profiler_log'  # Change this as needed
    profiler.start_trace(log_dir)
    try:
        # Calculate loss
        loss = mse_loss_one_batch(mat_u, mat_v, rows, columns, ratings)
        print(f"Loss: {loss}")
    finally:
        # Stop profiling
        profiler.stop_trace()

# Memory-efficient implementation of MSE loss
def mse_loss_memory_efficient(mat_u, mat_v, rows, columns, ratings):
    # Directly compute the predicted ratings
    predicted_ratings = jnp.sum(mat_u[rows] * mat_v[:, columns].T, axis=1)
    
    # Compute the MSE loss
    loss = jnp.mean((predicted_ratings - ratings) ** 2)
    return loss

# Testing the memory-efficient implementation
def test_memory_efficient_mse():
    # Sample user and item factor matrices
    mat_u = jax.random.normal(jax.random.PRNGKey(0), (4, 2))  # 4 users, 2 latent factors
    mat_v = jax.random.normal(jax.random.PRNGKey(1), (2, 4))  # 2 latent factors, 4 items

    # Sample indices and ratings
    rows = jnp.array([0, 1, 2, 3])
    columns = jnp.array([0, 1, 2, 3])
    ratings = jnp.array([5.0, 4.0, 3.0, 2.0])

    # Run memory inspection
    print("Running memory inspection with original function...")
    run_memory_inspection(mat_u, mat_v, rows, columns, ratings)

    # Test the memory-efficient implementation
    print("Testing memory-efficient MSE loss function...")
    loss_memory_efficient = mse_loss_memory_efficient(mat_u, mat_v, rows, columns, ratings)
    print(f"Memory-efficient Loss: {loss_memory_efficient}")

# Execute the test function
test_memory_efficient_mse()

Running memory inspection with original function...
Loss: 5.543153285980225
Testing memory-efficient MSE loss function...
Memory-efficient Loss: 32.878211975097656
